# 上週作業

In [ ]:
import os

os.chdir("../../")

In [ ]:
import json

from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

from src.io.path_definition import get_project_dir


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Ingredients: {input}\nOrigin: {output}",
)

with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

examples = []

for recipe in recipe_train:
    examples.append({"input": " ".join(recipe['ingredients']),
                     "output": recipe['cuisine']})

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=5,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Find the recipe origin based on the ingredients",
    suffix="Ingredients: {ingredients}\nOrigin:",
    input_variables=["ingredients"],
)

In [ ]:
similar_prompt

In [ ]:
with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_test.json'), 'r') as f:
    recipe_test = json.load(f)

existing_ingredients = recipe_test[99]['ingredients']

similar_prompt.invoke(", ".join(existing_ingredients))

In [ ]:
from langchain.chat_models import ChatOpenAI

from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

In [ ]:
chain = similar_prompt|model

In [ ]:
chain.invoke(", ".join(existing_ingredients))

# Remote server

### 1. Making a POST Request (發送 POST 請求):

- requests.post(...) sends an HTTP POST request to the specified URL.
- The URL "http://localhost:5000/openai/invoke" points to a local server running on port 5000, at the endpoint /openai/invoke.
- The json parameter is used to send a JSON payload with the request. In this case, the payload is {'input': "Where is Taiwan"}.
- requests.post(...) 發送一個 HTTP POST 請求到指定的 URL。
- URL "http://localhost:5000/openai/invoke" 指向一個本地服務器，該服務器在端口 5000 上運行，並且指向 /openai/invoke 端點。
- json 參數用於隨請求發送 JSON 負載。在這個例子中，負載是 {'input': "Where is Taiwan"}。

### 2. Response Handling (響應處理):

- The server processes the request and sends back a response.
- The response is stored in the response variable, which can then be inspected or used further in the code.
- 服務器處理請求並返回響應。
- 響應存儲在 response 變量中，之後可以檢查或在代碼中進一步使用。

In [ ]:
import requests

response = requests.post(
    "http://localhost:5000/openai/invoke",
    json={'input': "Where is Taiwan"}
)

In [ ]:
response.json()

# Use the remote model as Software as a service (SaaS)

## Basic Usage

### 1. Creating an Instance of RemoteRunnable (創建 RemoteRunnable 的實例):

- This line creates an instance of RemoteRunnable and initializes it with the URL of the remote language model service. In this case, the service is running locally on http://localhost:5000/openai/.
- 這行代碼創建一個 RemoteRunnable 的實例，並用遠程語言模型服務的 URL 進行初始化。在這個例子中，服務在本地運行，URL 為 http://localhost:5000/openai/。

In [ ]:
from langserve import RemoteRunnable

llm = RemoteRunnable("http://localhost:5000/openai/")

### 2. Asynchronous Streaming of Responses (異步流式處理回應):

- llm.astream("Where is Taiwan?") sends the query "Where is Taiwan?" to the remote service and retrieves the response as a stream.
- async for msg in ... is used to handle the streaming responses asynchronously.
- print(msg.content, end="", flush=True) prints each message content received from the stream without adding a new line after each message, and flushes the output buffer to ensure the message is displayed immediately.
- llm.astream("Where is Taiwan?") 將查詢 "Where is Taiwan?" 發送到遠程服務，並以流的形式檢索回應。
- async for msg in ... 用於異步處理流式回應。
- print(msg.content, end="", flush=True) 打印每個從流中接收到的消息內容，不在每個消息後添加新行，並刷新輸出緩衝區以確保消息立即顯示。

In [ ]:
# Supports astream
async for msg in llm.astream("Where is Taiwan?"):
    print(msg.content, end="", flush=True)

In [ ]:
output = llm.invoke("Where is Taiwan?")

In [ ]:
output.content

## Make the external service a part of the chain

### 1. Comedian Chain (喜劇演員鏈)

- ChatPromptTemplate.from_messages(...) creates a prompt template where the system prompt instructs the model to either tell a joke or state a fact, and the human prompt provides the input.
- This template is then piped (|) to a language model (llm) to generate the comedian's response.
- ChatPromptTemplate.from_messages(...) 創建一個提示模板，其中系統提示指示模型要麼講一個笑話，要麼陳述一個不搞笑的事實，並且僅輸出一個。
- 然後將此模板通過管道（|）傳遞給語言模型（llm），以生成喜劇演員的回應。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
            ('human', '{input}'
            )
        ]
    )
    | llm
)


### 2. Joke Classifier Chain

- This chain is similar to the comedian chain but serves a different purpose.
- The system prompt asks the model to classify the joke as "funny" or "not funny" and repeat the first five words for reference.
- This template is also piped to the language model (llm).
- 這個鏈與喜劇演員鏈類似，但用途不同。
- 系統提示要求模型將笑話分類為“搞笑”或“不搞笑”，並重複笑話的前五個詞以供參考。
- 此模板也通過管道傳遞給語言模型（llm）。

In [ ]:
joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | llm
)


### 3. Combining Chains with RunnablePassthrough

- This combines the comedian chain and the joke classifier chain using RunnablePassthrough.assign.
- The comedian chain generates the output, and then this output is passed to the joke classifier chain to classify its humor.
- 這將喜劇演員鏈和笑話分類器鏈結合在一起，使用 RunnablePassthrough.assign。
- 喜劇演員鏈生成輸出，然後將此輸出傳遞給笑話分類器鏈以分類其幽默性。

In [ ]:
chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [ ]:
chain.invoke({"input": "A man and a beer"})

# ChatBot

In [ ]:
from IPython.display import Image

Image(url="https://python.langchain.com/v0.1/assets/images/chat_use_case-eb8a4883931d726e9f23628a0d22e315.png")

- N-Shot
- The historical chat history can be consdiered as a list of question-answer pairs
- If the chatbot doesn’t remember past chats, it’s called stateless because it doesn’t know what happened before.

## Minimal Example

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

In [ ]:
model.invoke([HumanMessage(content="What did you just say?")])

The `memory` is not there, so it does not understand your question.

The following example shows how to add memory into the conversation.

In [ ]:
model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        ),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ]
)

## Prompt templates

### 1. Creating a ChatPromptTemplate 

- ChatPromptTemplate.from_messages(...) creates a prompt template for the chatbot.
- The first message in the template is a system message: "You are a helpful assistant. Answer all questions to the best of your ability." This message sets the context and behavior of the assistant, instructing it to be helpful and thorough in its responses.
- MessagesPlaceholder(variable_name="messages") is a placeholder for dynamic content. The variable_name="messages" specifies that this placeholder will be filled with user messages during the conversation.
- ChatPromptTemplate.from_messages(...) 創建了一個聊天機器人的提示模板。
- 模板中的第一條消息是一條系統消息：“You are a helpful assistant. Answer all questions to the best of your ability.” 此消息設置了助手的上下文和行為，指示其在回答中要提供幫助並盡力而為。
- MessagesPlaceholder(variable_name="messages") 是一個動態內容的佔位符。variable_name="messages" 指定該佔位符將在對話中插入用戶消息。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

### 2. Creating the Chain

- This line pipes (|) the prompt template to a language model (model).
- chain represents a sequence of operations where the prompt template is used to format user messages, and the language model processes these messages to generate responses.
- 這行代碼通過管道（|）將提示模板傳遞給語言模型（model）。
- chain 代表一系列操作，其中提示模板用於格式化用戶消息，語言模型處理這些消息以生成回應。

In [ ]:
chain = prompt | model

The MessagesPlaceholder above inserts chat messages passed into the chain's input as chat_history directly into the prompt. Then, we can invoke the chain like this:

In [ ]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

## Example of Using MessageHistory

As a shortcut for managing the chat history, we can use a MessageHistory class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called ChatMessageHistory

### 1. Importing the ChatMessageHistory Class (導入 ChatMessageHistory 類)

- This line imports the ChatMessageHistory class from the langchain.memory module. This class is used to handle the chat messages in memory.
- 這行代碼從 langchain.memory 模塊中導入 ChatMessageHistory 類。此類用於在內存中處理聊天消息。

In [ ]:
from langchain.memory import ChatMessageHistory

### 2. Creating an Instance of ChatMessageHistory (創建 ChatMessageHistory 的實例)

- This line creates an instance of ChatMessageHistory. This instance will store the chat messages in memory for this session.
- 這行代碼創建一個 ChatMessageHistory 的實例。該實例將在此會話期間將聊天消息存儲在內存中。

In [ ]:
demo_chat_history = ChatMessageHistory()

### 3. Adding User and AI Messages (添加用戶和 AI 消息)

- demo_chat_history.add_user_message("hi!") adds a user message ("hi!") to the chat history.
- demo_chat_history.add_ai_message("whats up?") adds an AI response ("whats up?") to the chat history.
- demo_chat_history.add_user_message("hi!") 將用戶消息（“hi!”）添加到聊天記錄中。
- demo_chat_history.add_ai_message("whats up?") 將 AI 回應（“whats up?”）添加到聊天記錄中。

In [ ]:
demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

demo_chat_history.messages

### 4. Retrieving the Messages (檢索消息)

- This line retrieves the list of messages stored in demo_chat_history. Each message is an object that contains information about the sender (user or AI) and the content of the message.
- 這行代碼檢索存儲在 demo_chat_history 中的消息列表。每條消息都是一個對象，包含有關發送者（用戶或 AI）和消息內容的信息。

In [ ]:
{"messages": demo_chat_history.messages}

In [ ]:
demo_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_chat_history.messages})

response

In [ ]:
# Put the response back into the demo_chat_history

demo_chat_history.add_ai_message(response)

demo_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_chat_history.messages})

# **** 預計第一個小時結束 ****

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from src.initialization import credential_init


credential_init()

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

## Conversational Retrievers - Step 1

- 土味情話反殺大全 (推薦上Youtube看)

In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

df = pd.DataFrame(data=[["确认过眼神，你是我爱的人。", "确认过眼神，我是你泡不到的人。"],
                         ["万水千山总是情，爱我多一点行不行。", "一寸光阴一寸金，劝你死了这条心。"],
                         ["今天吃了泡面，吃了炒面，还是想走进你的心里面。", "吃那么多面，最后还不是变成大便。"],
                         ["草莓，蓝莓，蔓越莓，今天你想我了没？", "冬瓜，西瓜，哈密瓜，你再巴巴我打得你叫妈妈。"],
                         ["众生皆苦，唯你独甜。", "尝遍众生，你为渣男代言。"],
                         ["你喜欢瑞士名表还是我帅气的外表？", "我喜欢去年买了个表。"],
                         ["我想问一条路，到哥哥心里的路。", "山路十八弯，走完脑血栓。"],
                         ["小姐姐，我心里给你留了一块地，死心塌地。", "对不起，我的心里只容得下一块地，玛莎拉蒂。"],
                         ["小姐姐你笑起来真好看啊。", "你看起来真好笑啊。"],
                         ["亲爱的你知道吗，你的笑容没有酒，我却醉得像条狗", "我的笑容没有酒，你是真的像条狗"],
                         ["宝贝儿，我在手上划了一道口子，你也划一下吧，这样我们就是两口子了", "我怕我们的血溶到一起，被你发现其实我是你爸爸"],
                         ["这世间万物都有尽头，落叶归根，而我归你", "对不起 我不收垃圾"],
                         ["请问……我想问一下路，那条通往你心里的路", "八格牙路"],
                         ["你今天怎么怪怪的？ 怪可爱的",  "你今天也怪怪的，怪恶心的"],
                         ["亲爱的，你知道我和唐僧的区别吗？ 唐僧取经我娶你", "知道你和沙僧的区别吗？ 他叫沙僧你叫沙雕"],
                         ["亲爱的，你不觉得累吗？ 你已经在我的脑海里跑了好几圈了", "傻孩子，我在找出口呢"],
                         ["莫文蔚的阴天。孙燕姿的雨天，周杰伦的晴天，都不如你和我聊天", "求求你了，能否还我一个宁静的夏天"],
                         ["如果你是方便面，那我就是白开水，今生今世，我泡定你了", "故事的最后，她变成了屎，你变成了尿，你们终究分道扬镳"],
                         ["大年三十晚上的鞭炮再响，也没有我想你那么想", "大年三十晚上的鞭炮再响，也没有你放的屁响"],
                         ["c罗可以上演帽子戏法，可我想你却没有办法", "c罗可以上演帽子戏法，我也可以给你上演绿帽子戏法"],
                         ["不要抱怨，抱我", "抱不起来，太重"],
                         ["你有没有发现我的眼睛很好看？因为我满眼都是你啊", "对不起，你眼睛在哪呢？"]], 
                  columns=['input', 'output'])

documents = []

for _, row in df.iterrows():
    documents.append(Document(page_content=row['input'], metadata={'output': row['output']}))

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 10})

## Build Chat Chain - Step 2

In [ ]:
from operator import itemgetter

from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate,  MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser


system_prompt = PromptTemplate.from_template("""You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                                                You will respond with the following style, cheesy pickup lines, shown in the context:\n\n{context}
                            
                                                You will reply in simplified Chinese (簡體中文). 
                                              """)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                        context to the question:
                                        {context}
                                        Question: {question}
                                       """
                                      )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_template = ChatPromptTemplate.from_messages([system_message,
                                                  MessagesPlaceholder(variable_name="messages"),
                                                  human_message
                                                  ])

chain = {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "messages": itemgetter("message")} | chat_template | model | StrOutputParser()

In [ ]:
# A cleaner way of writing the pipeline.

# chain = RunnablePassthrough.assign(context=itemgetter('question')|retriever) | chat_template | model | StrOutputParser()

## Test - Step 3

https://www.wenan.wang/qibaitiaotuweiqinghua.html

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory


chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    answer = chain.invoke({"question": question,
                                "message": chat_history.messages
                                })

    print(answer)
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer)

### 回家作業 2: 將retriever抽換成WikipediaRetriever

基本上，你可以將這個retriever的內容抽換成任何你需要的資料，來加快寫報告的效率。記得Double Check....

## Compress the chat history to reduce the size of the prompt


https://github.com/langchain-ai/langserve/blob/main/examples/conversational_retrieval_chain/server.py

### Condensation - Step 1

In [ ]:
system_prompt = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the 
                                                follow up question to be a standalone question, in its original language.
                                              """)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                       Question: {question}
                                       """)

human_message = HumanMessagePromptTemplate(prompt=human_prompt)


condensed_chat_template = ChatPromptTemplate.from_messages([system_message,
                                                  MessagesPlaceholder(variable_name="messages"),
                                                  human_message
                                                  ])

condensed_chain = {"question": itemgetter("question"),
                   "messages": itemgetter("message")} | condensed_chat_template | model | StrOutputParser()

## Retrieval - Step 2

How to implement this properly?

Let start from a higher point of view

{"standalone_question": condensed_chain}|RunnablePassthrough.assign(context=itemgetter('standalone_question')|retriever}

In [ ]:
from langchain_core.runnables import RunnablePassthrough


system_prompt = PromptTemplate.from_template("""You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                                                You will respond with the following style, cheesy pickup lines, shown in the context:\n\n{context}
                            
                                                You will reply in simplified Chinese (簡體中文). 
                                              """)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                        context to the question:
                                        {context}
                                        Question: {standalone_question}
                                        """
                                      )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)


retrieval = RunnablePassthrough.assign(context=itemgetter('standalone_question')|retriever)

retrieval_template = ChatPromptTemplate.from_messages([system_message,
                                                        human_message
                                                       ])

retrieval_chain = retrieval|retrieval_template|model|StrOutputParser()

In [ ]:
final_chain = {"standalone_question": condensed_chain}|retrieval_chain

In [ ]:
chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    answer = final_chain.invoke({"question": question,
                                 "message": chat_history.messages
                                })

    print(answer)
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer)